In [1]:
import tensorflow as tf

In [2]:
tf.executing_eagerly()
r = tf.Variable(10.0, name='r')
r

<tf.Variable 'r:0' shape=() dtype=float32, numpy=10.0>

In [3]:
def area(r):
    circle = 3.14 * (r ** 2.00)
    return circle

area(10)

314.0

In [4]:
print("Area of respective circle: %2.2f, %2.2f, %2.2f" % (area(tf.constant(50.0)), area(tf.constant(100.0)), area(tf.constant(150.0))))

Area of respective circle: 7850.00, 31400.00, 70650.00


In [5]:
print(tf.autograph.to_code(area))

def tf__area(r):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('area', 'area_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as area_scope:
    circle = 3.14 * r ** 2.0
    do_return = True
    retval_ = area_scope.mark_return_value(circle)
  do_return,
  return ag__.retval(retval_)



In [6]:
tf_area = tf.autograph.to_graph(area)
tf_area

<function __main__.create_converted_entity_factory.<locals>.create_converted_entity.<locals>.tf__area(r)>

In [7]:
with tf.Graph().as_default():
    a1 = tf_area(tf.constant(50.0))
    a2 = tf_area(tf.constant(100.0))
    a3 = tf_area(tf.constant(150.0))
    
    with tf.compat.v1.Session() as sess:
        print("Area of respective circle: %2.2f, %2.2f, %2.2f" % (sess.run(a1), sess.run(a2), sess.run(a3)))

Area of respective circle: 7850.00, 31400.00, 70650.00


<h1>AutoGraph using tf.function<\h1>

In [8]:
r = tf.Variable(10.0, name="r")

In [9]:
@tf.function
def area(r):
    circle = 3.14 * (r ** 2.00)
    return circle

In [10]:
print(tf.autograph.to_code(area.python_function))

def tf__area(r):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('area', 'area_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as area_scope:
    circle = 3.14 * r ** 2.0
    do_return = True
    retval_ = area_scope.mark_return_value(circle)
  do_return,
  return ag__.retval(retval_)



In [11]:
area(60)

<tf.Tensor: id=52, shape=(), dtype=float32, numpy=11304.0>

In [12]:
print(area(tf.constant(50.0)), area(tf.constant(100.0)), area(tf.constant(150.0)))

tf.Tensor(7850.0005, shape=(), dtype=float32) tf.Tensor(31400.002, shape=(), dtype=float32) tf.Tensor(70650.0, shape=(), dtype=float32)


In [13]:
import traceback
import contextlib

In [14]:
# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print("Caught expected exception \n {}:".format(error_class))
        traceback.print_exc(limit=2)
        
    except Exception as e:
        raise(e)
        
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(error_class))

In [15]:
@tf.function
def add(a, b):
    return a + b

add(tf.ones([2, 3]), tf.ones([2, 3]))

<tf.Tensor: id=80, shape=(2, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>

In [16]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
print(tape.gradient(result, v))

tf.Tensor(1.0, shape=(), dtype=float32)


In [17]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])

# warm up
conv_layer(image); conv_fn(image)

print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

Eager conv: 1.1181732019995252
Function conv: 1.360472596999898
Note how there's not much difference in performance for convolutions


<h1>Tracing<\h1>

In [18]:
@tf.function
def double(a):
    print("Tracing with ", a)
    return a + a


print(double(tf.constant(1)), "\n")
print(double(tf.constant(1.1)), "\n")
print(double(tf.constant("a")))

Tracing with  Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32) 

Tracing with  Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32) 

Tracing with  Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)


In [19]:
print(double(tf.constant("b")))

tf.Tensor(b'bb', shape=(), dtype=string)


In [20]:
# print(double.pretty_printed_concrete_signatures())

In [21]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.constant("a"))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(tf.constant("b")))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [22]:
print(double_strings)

In [23]:
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Caught expected exception 
 <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-14-ac6cf5be5446>", line 5, in assert_raises
    yield
  File "<ipython-input-23-6ba07b14b756>", line 2, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_216 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_216]


In [24]:
@tf.function(input_signature=(tf.TensorSpec(shape=None, dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with ", x)
    return tf.where(x % 2 == 0 // 2, 3 * x + 1)

# print(next_collatz(tf.constant([1, 2])))
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))
    
    
with assert_raises(ValueError):
    next_collatz(tf.constant([1.0, 2.0]))

Tracing with  Tensor("x:0", dtype=int32)
Caught expected exception 
 <class 'ValueError'>:
Caught expected exception 
 <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-14-ac6cf5be5446>", line 5, in assert_raises
    yield
  File "<ipython-input-24-22760ee45afc>", line 8, in <module>
    next_collatz(tf.constant([[1, 2], [3, 4]]))
ValueError: in converted code:

    <ipython-input-24-22760ee45afc>:4 next_collatz  *
        return tf.where(x % 2 == 0 // 2, 3 * x + 1)
    /home/vegas/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:3801 where_v2
        raise ValueError("x and y must both be non-None or both be None.")

    ValueError: x and y must both be non-None or both be None.

Traceback (most recent call last):
  File "<ipython-input-14-ac6cf5be5446>", line 5, in assert_raises
    yield
  File "<ipython-input-24-22760ee45afc>", line 12, in <module>
    next_collatz(tf.constant([1.0, 2.0]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2.], shape=(2,), dtype=float32))
  input_signature: (
    TensorSp

In [25]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def g(x):
    print('Tracing with', x)
    return x

# No retrace!
print(g(tf.constant([1, 2, 3])))
print(g(tf.constant([1, 2, 3, 4, 5])))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


In [26]:
external_var = tf.Variable(0)

@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var: ", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator=iterator)
buggy_consume_next(iterator=iterator)
buggy_consume_next(iterator=iterator)

Value of external_var:  0
Value of external_var:  0
Value of external_var:  0


In [27]:
# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))

[0.91226089 0.908860445 0.281906247 0.561953306 0.636376]
[0.722215712 0.720584929 0.274668425 0.509425282 0.562426925]
[0.618279874 0.617271483 0.267963409 0.469497263 0.509775877]
[0.549929261 0.54922539 0.261728764 0.437793016 0.469770521]
[0.500467181 0.499939412 0.255911738 0.411813378 0.438013911]
[0.462484479 0.462069511 0.250467688 0.390011311 0.411996812]
[0.432106972 0.431769431 0.245358229 0.371369958 0.390166879]
[0.407080591 0.406799 0.240550384 0.355189413 0.371504068]
[0.385991037 0.385751367 0.236015543 0.340969741 0.355306566]
[0.367899 0.367691755 0.231728747 0.328342855 0.341073245]
[0.352152586 0.35197106 0.227668136 0.317030966 0.328435183]
[0.338283181 0.338122427 0.223814428 0.306819826 0.317114]
[0.325943828 0.325800151 0.220150635 0.297541201 0.306895047]
[0.31487143 0.314741969 0.216661632 0.289060831 0.297609776]
[0.304862291 0.304744869 0.21333389 0.281270146 0.289123684]
[0.295755923 0.295648783 0.210155353 0.274080217 0.281328052]
[0.287423879 0.287325621 

<tf.Tensor: id=310, shape=(5,), dtype=float32, numpy=
array([0.20926557, 0.20922782, 0.17145708, 0.20118028, 0.20398428],
      dtype=float32)>

In [28]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(x):
      ag__.converted_call(tf.print, f_scope.callopts, (x,), None, f_scope)
      x = ag__.converted_call(tf.tanh, f_scope.callopts, (x,), None, f_scope)
      return x,

    def loop_test(x):
      return ag__.converted_call(tf.reduce_sum, f_scope.callopts, (x,), None, f_scope) > 1
    x, = ag__.while_stmt(loop_test, loop_body, get_state, set_state, (x,), ('x',), ())
    do_return = True
    retval_ = f_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)

